In [1]:
from module import td3

import numpy as np
import random

import tensorflow as tf
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential

from collections import deque

import gym

# https://www.gymlibrary.dev/environments/classic_control/pendulum/

In [2]:
env = gym.make('Pendulum-v1', g=9.81)

# Get the state and action sizes
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]

print(state_size, action_size)

3 1


In [3]:
state = env.reset()
state = np.reshape(state[0], [1, state_size])
done = False
score = 0
for i in range(100):
    env.render()
    # select action
    action = np.random.uniform(low=-2, high=2, size=(1,))
    # perform the action
    next_state, reward, done, _, _= env.step(action)
    # update the score
    score += reward
    # move to the next state
    next_state = np.reshape(next_state, [1, state_size])
    state = next_state

score

/Users/sondinhvan/miniconda3/envs/rl_env/lib/python3.9/site-packages/gym/envs/classic_control/pendulum.py:167: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("Pendulum-v1", render_mode="rgb_array")
  gym.logger.warn(
/Users/sondinhvan/miniconda3/envs/rl_env/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


-578.729138331696

In [4]:
# training
env = gym.make('Pendulum-v1')
agent = td3.TD3Agent(state_size, action_size, batch_size=32,
                training_start=200, update_period=1,
                lr_critic=0.001, lr_actor=0.001,
                min_action=-1, max_action=1)
score_history = []
avg_score_history = []
n_episodes = 20000

for i in range(n_episodes):
    done = False
    truncated = False
    score = 0
    state = env.reset()
    state = np.reshape(state[0], [1, state_size])
    while not truncated and not done:
        action = agent.act(state, use_noise=True, noise_label='Gaussian')
        next_state, reward, done, truncated, _ = env.step(2 * action)
        next_state = next_state.reshape(1, state_size)
        agent.store_data(state, action, reward, next_state, done)
        agent.learn()
        state = next_state
        score += reward
        
    score_history.append(score)
    avg_score = np.mean(score_history[-40:])
    avg_score_history.append(avg_score)
    print('Episode: ', i, '- Score: ', round(score, 3), '- Average score: ', round(avg_score, 3), '- Memory size: ', len(agent.memory))

Episode:  0 - Score:  -1327.316 - Average score:  -1327.316 - Memory size:  200
Collect enough samples, training starting
Episode:  1 - Score:  -1380.089 - Average score:  -1353.703 - Memory size:  400
Episode:  2 - Score:  -1423.071 - Average score:  -1376.825 - Memory size:  600
Episode:  3 - Score:  -1853.123 - Average score:  -1495.9 - Memory size:  800
Episode:  4 - Score:  -1636.5 - Average score:  -1524.02 - Memory size:  1000
Episode:  5 - Score:  -1425.53 - Average score:  -1507.605 - Memory size:  1200
Episode:  6 - Score:  -1476.412 - Average score:  -1503.149 - Memory size:  1400
Episode:  7 - Score:  -1284.598 - Average score:  -1475.83 - Memory size:  1600
Episode:  8 - Score:  -1364.748 - Average score:  -1463.487 - Memory size:  1800
Episode:  9 - Score:  -1256.018 - Average score:  -1442.74 - Memory size:  2000
Episode:  10 - Score:  -928.805 - Average score:  -1396.019 - Memory size:  2200
Episode:  11 - Score:  -894.548 - Average score:  -1354.23 - Memory size:  2400

KeyboardInterrupt: 

In [ ]:
env = gym.make('Pendulum-v1', render_mode='human')
state = env.reset()
state = np.reshape(state[0], [1, state_size])
done = False
truncated = False
score = 0
while not done and not truncated:
    action = agent.act(state, use_noise=False)
    next_state, reward, done, truncated, _ = env.step(2 * action)
    next_state = next_state.reshape(1, state_size)
    state = next_state
    score += reward

print(score)